In [1]:
import pandas as pd
import numpy as np
import re
import os

In [2]:
def auto_count(months):

    def read(csv, weeks):
        month = pd.read_csv(csv)
        month.columns = month.iloc[0]
        month = month.iloc[::2]
        month = month.iloc[1:,:]
        return month
    
    def count(weeks, duty, lunch):
        lunchpat = r"Lunch 12[ ]*-[ ]*1[pm| ]*:[ ]*([A-Za-z]+)"
        dutypats = [r"4:30[pm| ]*-[ ]*9[pm| ]*:[ ]*([A-Za-z]+)", 
                       r"9[am| ]*-[ ]*1[pm| ]*:[ ]*([A-Za-z]+)", 
                       r"1[pm| ]*-[ ]*5[pm| ]*:[ ]*([A-Za-z]+)", 
                       r"5[pm| ]*-[ ]*9[pm| ]*:[ ]*([A-Za-z]+)"]
        for day in weeks:
            if day != "":
                if re.match(lunchpat, day) != None:
                    add_count(re.findall(lunchpat, day)[0], lunch)
                for dut in list(dutypats):
                    if re.match(dut, day) != None:
                        add_count(re.findall(dut, day)[0], duty)
                        break
                        
    def add_count(name, counts):
        try:
            counts[name] += 1
        except:
            counts[name] = 1
    
    duty = {}
    lunch = {}
    monthdfs = []
    weeks = []
    
    for i in months:
        monthdfs.append(read(i, weeks))
        
    monthdf = pd.concat(monthdfs)
    duties = sum(monthdf.values.tolist(), weeks)
    duties = "\n".join(str(x) for x in duties if pd.isna(x) == False)
    duties = duties.split("\n")
    
    count(duties, duty, lunch)
    
    d = pd.DataFrame.from_dict(duty, orient = "index")
    d = d.rename(columns={d.columns[0]: 'Duty'})
    l = pd.DataFrame.from_dict(lunch, orient = "index")
    l = l.rename(columns={l.columns[0]: 'Lunch'})
    
    return pd.concat([d, l], axis = 1).fillna(0).astype(int)

In [3]:
def combine(name, repeat, df):
    df.loc[name, "Duty"] += df.loc[repeat, "Duty"]
    df.loc[name, "Lunch"] += df.loc[repeat, "Lunch"]
    df.drop(repeat, inplace = True)
def remove(name, df):
    df.drop(name, inplace = True)
def addl(name, shifts, df):
    df.loc[name, "Lunch"] += shifts

In [4]:
cwd = os.getcwd()
files = os.listdir(cwd)

months = [i for i in files if i[0] == "2"]
    
ds = auto_count(months)
combine("Jenn", "Jen", ds)
combine("Alex", "Alexis", ds)
remove("Emilio", ds)

In [5]:
ds["Total Hours"] = (ds["Duty"] * 4) + ds["Lunch"]
ds.sort_values("Total Hours", ascending = False)

,Duty,Lunch,Total Hours
Rachel,15,8,68
Alex,15,6,66
Kanishk,15,6,66
Jade,15,5,65
Jessie,14,9,65
CJ,15,5,65
JJ,14,8,64
Eden,14,8,64
Savannah,14,7,63
Luzia,14,7,63
